In [2]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\ITSC\Desktop\project\DL\data\nsmc\ratings.txt", sep = "\t")

In [4]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [5]:
df.shape

(200000, 3)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [7]:
# document 에 결측치 있어서 결측치 날림 
df = df.dropna()

In [8]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [9]:
df.shape

(199992, 3)

In [10]:
komoran = Komoran()

In [11]:
%%time
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
docs = df["document"].map(lambda x: komoran.nouns(x))

CPU times: total: 1min 52s
Wall time: 1min 28s


In [12]:
docs

0                                                       [때]
1         [디자인, 학생, 외국, 디자이너, 전통, 발전, 문화, 산업, 사실, 우리나라, ...
2                                    [폴리스, 스토리, 시리즈, 뉴, 최고]
3                                [연기, 것, 라고, 생각, 몰입, 영, 화지]
4                                        [안개, 밤하늘, 초승달, 영화]
                                ...                        
199995                                         [포켓, 몬스터, 짜]
199996                                                   []
199997                            [완전, 사이코, 영화, 마지막, 영화, 질]
199998                                          [라따뚜이, 스머프]
199999                       [포, 풍, 저그, 가나, 신다영, 차영, 차영, 차]
Name: document, Length: 199992, dtype: object

In [13]:
df.iloc[-1]

id                  8548411
document    포풍저그가나가신다영차영차영차
label                     0
Name: 199999, dtype: object

In [14]:
# word2vec 모델 학습
model = Word2Vec(sentences = docs, vector_size = 200, window = 4, min_count = 2, sg = 1)

- word2vec의 주요 하이퍼파라미터
    - sentences : Word2Vec 모델 학습에 필요한 문장 데이터
    - vector_size : 단어 임베딩 벡터의 차원(크기)
    - window : 주변 단어 윈도우의 크기
    - min_count : 단어 최소 빈도 수 제한(설정된 min_count 빈도 수 미만의 단어들은 학습하지 않음)
    - sg : 0(CBOW 모델), 1(skip-gram 모델)

In [15]:
# 모델 저장
model.save("./model/nscm_w2v.model")

In [16]:
# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count :", model.corpus_count)
print("corpus_total_words :", model.corpus_total_words)

corpus_count : 199992
corpus_total_words : 1076840


# word2vec 모델 활용

In [17]:
# 모델 로딩
model = Word2Vec.load("./model/nscm_w2v.model")
print(model.corpus_total_words)

1076840


In [18]:
# '사랑' 이란 단어로 생성한 단어 임베딩 벡터
print("사랑 : ", model.wv["사랑"])

사랑 :  [ 0.04706824 -0.38106966  0.0888775   0.32955068 -0.18055649 -0.26025057
  0.18544894  0.04895185 -0.00780015  0.4085669  -0.16917202 -0.04639477
  0.38053504 -0.07704426  0.06779956  0.5148434   0.17888618 -0.01086047
 -0.2557851  -0.411493    0.12976944  0.2914647   0.04696575  0.13756308
 -0.2625612  -0.05060172  0.02457256 -0.3017246  -0.28581586 -0.3403364
  0.09617167  0.26253548  0.00334849  0.27580044 -0.07122608 -0.03869795
 -0.18215679  0.02767465 -0.27021125 -0.09391867 -0.29149356  0.01545098
 -0.00684852 -0.15330562  0.49919236  0.40387347 -0.30812994 -0.01883363
  0.292169    0.16339362  0.31852362 -0.2726235  -0.15184432  0.15778421
 -0.19326141 -0.10329254 -0.10540201 -0.30437198 -0.24932384  0.04926739
  0.13109301  0.15502812 -0.32905513  0.28846022 -0.24275947  0.06192928
 -0.26419365  0.21739236 -0.24496032  0.8549031   0.10579114 -0.04742035
  0.15520413  0.09464101  0.28953686 -0.45625228  0.22787748 -0.5045905
 -0.34441966 -0.31505743 -0.16339633 -0.0074687

- 모델을 학습할 때 설정한 vector_size 하이퍼파라미터 만큼 단어 임베딩 벡터 차원 크기가 결정됨

- model.wv.similarity() 함수 : 두 단어간의 유사도를 계산

In [19]:
# 단어 유사도 계산
print("일요일 = 월요일", model.wv.similarity(w1 = "일요일", w2 = "월요일"), sep = "\t")

일요일 = 월요일	0.9072058


In [20]:
print("대기업 = 삼성", model.wv.similarity(w1 = "대기업", w2 = "삼성"), sep = "\t")

대기업 = 삼성	0.86884606


In [21]:
print("일요일 = 삼성", model.wv.similarity(w1 = "일요일", w2 = "삼성"), sep = "\t")

일요일 = 삼성	0.6464495


- model.wv.most_similar() 함수 : 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환
    - 벡터 공간에서 가장 가까운 거리에 있는 단어들을 반환
    - topn : 반환되는 유사한 단어 수
- 유사도가 1에 가까울수록 두 단어는 동일한 의미이거나 문법적으로 관련이 있을 가능성이 높음

In [22]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("시리즈", topn = 5))

[('엑스맨', 0.8242610096931458), ('포터', 0.8075320720672607), ('해리', 0.7929564118385315), ('스타워즈', 0.7901511192321777), ('반지의 제왕', 0.7891226410865784)]


In [24]:
print(model.wv.most_similar("왕", topn = 5))

[('끝판', 0.8201351165771484), ('죤', 0.7494338750839233), ('흥', 0.7432554960250854), ('사마', 0.7415783405303955), ('지대', 0.7407092452049255)]


In [25]:
print(model.wv.most_similar("비빔밥", topn = 5))

[('누더기', 0.9807769060134888), ('뒷수습', 0.9802371859550476), ('몰골', 0.9781771898269653), ('헨젤과 그레텔', 0.9780308604240417), ('코미디 드라마', 0.977856457233429)]


In [26]:
import gensim

In [27]:
sentences = gensim.models.word2vec.Text8Corpus("./data/text8.txt")

In [28]:
model = Word2Vec(sentences, vector_size = 200)

In [29]:
model.wv.most_similar(positive = ['woman', 'king'], negative = ['man'])

[('queen', 0.6189376711845398),
 ('throne', 0.5696812868118286),
 ('princess', 0.5456494092941284),
 ('empress', 0.5442882776260376),
 ('prince', 0.5375813841819763),
 ('elizabeth', 0.5322781205177307),
 ('aragon', 0.5179737210273743),
 ('emperor', 0.5175638198852539),
 ('consort', 0.5153141021728516),
 ('isabella', 0.5114498138427734)]